# Market making backtests

In this notebook I automate the generation of market making backtests for the thesis.

### Initial setup

In [ ]:
import os
import pickle
import time

import pandas as pd
import polars as pl
import numpy as np

from datetime import datetime

from lob.exchange import Exchange
from lob.traders import PureMarketMaker
from lob.commissions import BitCommissions
from lob.plots import set_plot_style
from lob.utils import get_lot_size, get_tick_size, ensure_dir_exists
from rl.utils import send_notification

In [ ]:
# Configure Polars 
cfg = pl.Config()
cfg.set_tbl_rows(20)

# Configure plotting
set_plot_style()

In [ ]:
# Define custom colors
color_green = "#13961a"
color_red = "#eb5c14"

In [ ]:
# Set random seed
SEED = 1

### Pure market makers (volume 100)

In this section I generate the statistics of the pure market making strategy with multiple priorities.

In [ ]:
TS_START = pd.Timestamp("2023-09-01 00:00:00") # Start of the episode
TS_END = pd.Timestamp("2023-09-13 23:59:59") # End of the episode

In [ ]:
# Set the parameters
EXCHANGE_NAME = "BIT.COM" 
SYMBOL = "SOL-USDT"
PATH = "~/Projects/thesis-market-making/reinforcement-learning/data/"
TICK_SIZE = get_tick_size(EXCHANGE_NAME) # Tick size of the limit order book
LOT_SIZE = get_lot_size(EXCHANGE_NAME) # Lot size of the limit order book
DEPTH = 20 # Depth of the data to load to the limit order book (max 20)
EXCHANGE_TRADER_ID = "Exchange"
MAX_STEPS = None # Maximum number of steps in an episode
WIN = 0 # Window size for the features computation
# LOGGING = False # Indicates whether to log events
LOGGING = True
TS_SAVE = datetime.now().strftime("%Y-%m-%d_%H-%M-%S") # Ts for model saving
RNG = np.random.default_rng(seed=SEED) # Random number generator

# Set the parameters for the stochastic backtest
LATENCY_COMP_PARAMS = {
    0: {"prob": 0.9, "divisor": 1},
    1: {"prob": 0.9, "divisor": 1},
    2: {"prob": 0.9, "divisor": 1},
    3: {"prob": 0.9, "divisor": 1},
} # Latency compensation parameters for the stochastic backtest

In [ ]:
# Set the parameters for the automated backtest
priorities = [0, 1, 2, 3]
volumes = [100]

In [ ]:
# Initialize the results dictionary
results = {}

# Run the backtests
for priority in priorities:
    for volume in volumes: 

        # Initialize the limit order book and traders
        start = time.time()
        traders = {}

        # Pure market making strategy
        trader_id = f"PMM_prior_{priority}_vol_{volume}"
        inventory_manage = True
        description = f"Pure market maker with priority {priority} and volume {volume}."
        
        # Set the commission model
        if volume == 100:
            if priority == 0:
                tier = 5
            elif priority == 1:
                tier = 5
            elif priority == 2:
                tier = 2
            elif priority == 3:
                tier = 1
        elif volume == 10:
            if priority == 0:
                tier = 4
            elif priority == 1:
                tier = 3
            elif priority == 2:
                tier = 1
            elif priority == 3:
                tier = 1
        
        com_model = BitCommissions(tier=tier)
        trader = PureMarketMaker(
                trader_id,
                com_model=com_model,
                volume=volume,
                priority=priority,
                inventory_manage=inventory_manage,
            )
        traders[trader.id] = trader

        # Initialize the exchange
        exchange = Exchange(
            exchange_name=EXCHANGE_NAME,
            symbol_name=SYMBOL,
            tick_size=TICK_SIZE,
            lot_size=LOT_SIZE,
            depth=DEPTH,
            traders=traders,
            max_steps=MAX_STEPS,
            ts_start=TS_START,
            ts_end=TS_END,
            win=WIN,
            path=PATH,
            rl_trader_id="",
            latency_comp_params=LATENCY_COMP_PARAMS,
            logging=LOGGING,
            ts_save=TS_SAVE,
            description=description,
            rng=RNG,
            )
        end = round(time.time() - start, 2)

        # Run the exchange simulation
        start = time.time()
        exchange.run()
        end = round(time.time() - start, 2)

        # Save the results
        timestamps = exchange.stats["ts"]
        trader_stats = traders[trader_id].stats
        initial_cost = 20.5 * volume * 2
        results[trader_id] = {
            "timestamps": timestamps,
            "trader_stats": trader_stats,
            "initial_cost": initial_cost,
        }
        
send_notification(message="Backtest finished!", time=20000)

In [ ]:
# Save the results to a pickle file
save_dir = "automated_backtests"

ensure_dir_exists(save_dir)
save_path = os.path.join(save_dir, f"results_{TS_SAVE}.pickle")
with open(save_path, "wb") as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
print(f"Results saved to {save_path}.")

### Pure market makers (volume 10)

In this section I generate the statistics of the pure market making strategy with multiple priorities.

In [ ]:
TS_START = pd.Timestamp("2023-09-01 00:00:00") # Start of the episode
TS_END = pd.Timestamp("2023-09-13 23:59:59") # End of the episode

In [ ]:
# Set the parameters
EXCHANGE_NAME = "BIT.COM" 
SYMBOL = "SOL-USDT"
PATH = "~/Projects/thesis-market-making/reinforcement-learning/data/"
TICK_SIZE = get_tick_size(EXCHANGE_NAME) # Tick size of the limit order book
LOT_SIZE = get_lot_size(EXCHANGE_NAME) # Lot size of the limit order book
DEPTH = 20 # Depth of the data to load to the limit order book (max 20)
EXCHANGE_TRADER_ID = "Exchange"
MAX_STEPS = None # Maximum number of steps in an episode
WIN = 0 # Window size for the features computation
# LOGGING = False # Indicates whether to log events
LOGGING = True
TS_SAVE = datetime.now().strftime("%Y-%m-%d_%H-%M-%S") # Ts for model saving
RNG = np.random.default_rng(seed=SEED) # Random number generator

# Set the parameters for the stochastic backtest
LATENCY_COMP_PARAMS = {
    0: {"prob": 0.9, "divisor": 1},
    1: {"prob": 0.9, "divisor": 1},
    2: {"prob": 0.9, "divisor": 1},
    3: {"prob": 0.9, "divisor": 1},
} # Latency compensation parameters for the stochastic backtest

In [ ]:
# Set the parameters for the automated backtest
priorities = [0, 1, 2, 3]
volumes = [10]

In [ ]:
# Initialize the results dictionary
results = {}

# Run the backtests
for priority in priorities:
    for volume in volumes: 

        # Initialize the limit order book and traders
        start = time.time()
        traders = {}

        # Pure market making strategy
        trader_id = f"PMM_prior_{priority}_vol_{volume}"
        inventory_manage = True
        description = f"Pure market maker with priority {priority} and volume {volume}."
        
        # Set the commission model
        if volume == 100:
            if priority == 0:
                tier = 5
            elif priority == 1:
                tier = 5
            elif priority == 2:
                tier = 2
            elif priority == 3:
                tier = 1
        elif volume == 10:
            if priority == 0:
                tier = 4
            elif priority == 1:
                tier = 3
            elif priority == 2:
                tier = 1
            elif priority == 3:
                tier = 1
        
        com_model = BitCommissions(tier=tier)
        trader = PureMarketMaker(
                trader_id,
                com_model=com_model,
                volume=volume,
                priority=priority,
                inventory_manage=inventory_manage,
            )
        traders[trader.id] = trader

        # Initialize the exchange
        exchange = Exchange(
            exchange_name=EXCHANGE_NAME,
            symbol_name=SYMBOL,
            tick_size=TICK_SIZE,
            lot_size=LOT_SIZE,
            depth=DEPTH,
            traders=traders,
            max_steps=MAX_STEPS,
            ts_start=TS_START,
            ts_end=TS_END,
            win=WIN,
            path=PATH,
            rl_trader_id="",
            latency_comp_params=LATENCY_COMP_PARAMS,
            logging=LOGGING,
            ts_save=TS_SAVE,
            description=description,
            rng=RNG,
            )
        end = round(time.time() - start, 2)

        # Run the exchange simulation
        start = time.time()
        exchange.run()
        end = round(time.time() - start, 2)

        # Save the results
        timestamps = exchange.stats["ts"]
        trader_stats = traders[trader_id].stats
        initial_cost = 20.5 * volume * 2
        results[trader_id] = {
            "timestamps": timestamps,
            "trader_stats": trader_stats,
            "initial_cost": initial_cost,
        }
        
send_notification(message="Backtest finished!", time=20000)

In [ ]:
# Save the results to a pickle file
save_dir = "automated_backtests"

ensure_dir_exists(save_dir)
save_path = os.path.join(save_dir, f"results_{TS_SAVE}.pickle")
with open(save_path, "wb") as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
print(f"Results saved to {save_path}.")

### Pure market maker (50 seeds)

In this section I generate the statistics of the pure market making strategy with multiple priorities.

In [ ]:
TS_START = pd.Timestamp("2023-09-11 00:00:00") # Start of the episode
TS_END = pd.Timestamp("2023-09-13 23:59:59") # End of the episode

In [ ]:
# Set the parameters
EXCHANGE_NAME = "BIT.COM" 
SYMBOL = "SOL-USDT"
PATH = "~/Projects/thesis-market-making/reinforcement-learning/data/"
TICK_SIZE = get_tick_size(EXCHANGE_NAME) # Tick size of the limit order book
LOT_SIZE = get_lot_size(EXCHANGE_NAME) # Lot size of the limit order book
DEPTH = 20 # Depth of the data to load to the limit order book (max 20)
EXCHANGE_TRADER_ID = "Exchange"
MAX_STEPS = None # Maximum number of steps in an episode
WIN = 0 # Window size for the features computation
# LOGGING = False # Indicates whether to log events
LOGGING = False
TS_SAVE = datetime.now().strftime("%Y-%m-%d_%H-%M-%S") # Ts for model saving


# Set the parameters for the stochastic backtest
LATENCY_COMP_PARAMS = {
    0: {"prob": 0.9, "divisor": 1},
    1: {"prob": 0.9, "divisor": 1},
    2: {"prob": 0.9, "divisor": 1},
    3: {"prob": 0.9, "divisor": 1},
} # Latency compensation parameters for the stochastic backtest

In [ ]:
# Set the parameters for the automated backtest
priorities = [1]
volumes = [100]

In [ ]:
# Initialize the results dictionary
results = {}

# Run the backtests
for seed in range(1, 51):
    for priority in priorities:
        for volume in volumes: 
            RNG = np.random.default_rng(seed=seed)

            # Initialize the limit order book and traders
            start = time.time()
            traders = {}

            # Pure market making strategy
            trader_id = f"PMM_prior_{priority}_vol_{volume}_{seed}"
            inventory_manage = True
            description = f"Pure market maker with priority {priority} and volume {volume}."
            
            # Set the commission model
            if volume == 100:
                if priority == 0:
                    tier = 5
                elif priority == 1:
                    tier = 5
                elif priority == 2:
                    tier = 2
                elif priority == 3:
                    tier = 1
            elif volume == 10:
                if priority == 0:
                    tier = 4
                elif priority == 1:
                    tier = 3
                elif priority == 2:
                    tier = 1
                elif priority == 3:
                    tier = 1
            
            com_model = BitCommissions(tier=tier)
            trader = PureMarketMaker(
                    trader_id,
                    com_model=com_model,
                    volume=volume,
                    priority=priority,
                    inventory_manage=inventory_manage,
                )
            traders[trader.id] = trader

            # Initialize the exchange
            exchange = Exchange(
                exchange_name=EXCHANGE_NAME,
                symbol_name=SYMBOL,
                tick_size=TICK_SIZE,
                lot_size=LOT_SIZE,
                depth=DEPTH,
                traders=traders,
                max_steps=MAX_STEPS,
                ts_start=TS_START,
                ts_end=TS_END,
                win=WIN,
                path=PATH,
                rl_trader_id="",
                latency_comp_params=LATENCY_COMP_PARAMS,
                logging=LOGGING,
                ts_save=TS_SAVE,
                description=description,
                rng=RNG,
                )
            end = round(time.time() - start, 2)

            # Run the exchange simulation
            start = time.time()
            exchange.run()
            end = round(time.time() - start, 2)

            # Save the results
            timestamps = exchange.stats["ts"]
            trader_stats = traders[trader_id].stats
            initial_cost = 20.5 * volume * 2
            results[trader_id] = {
                "timestamps": timestamps,
                "trader_stats": trader_stats,
                "initial_cost": initial_cost,
            }
        
send_notification(message="Backtest finished!", time=20000)

In [ ]:
# Save the results to a pickle file
save_dir = "automated_backtests"

ensure_dir_exists(save_dir)
save_path = os.path.join(save_dir, f"results_{TS_SAVE}.pickle")
with open(save_path, "wb") as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
print(f"Results saved to {save_path}.")

### AIRL market maker (50 seeds)

In this section I generate the statistics of the AIRL market making strategy with multiple priorities.

In [ ]:
TS_START = pd.Timestamp("2023-09-11 00:00:00") # Start of the episode
TS_END = pd.Timestamp("2023-09-13 23:59:59") # End of the episode

In [ ]:
# Set the parameters
EXCHANGE_NAME = "BIT.COM" 
SYMBOL = "SOL-USDT"
PATH = "~/Projects/thesis-market-making/reinforcement-learning/data/"
TICK_SIZE = get_tick_size(EXCHANGE_NAME) # Tick size of the limit order book
LOT_SIZE = get_lot_size(EXCHANGE_NAME) # Lot size of the limit order book
DEPTH = 20 # Depth of the data to load to the limit order book (max 20)
EXCHANGE_TRADER_ID = "Exchange"
MAX_STEPS = None # Maximum number of steps in an episode
WIN = 0 # Window size for the features computation
# LOGGING = False # Indicates whether to log events
LOGGING = False
TS_SAVE = datetime.now().strftime("%Y-%m-%d_%H-%M-%S") # Ts for model saving


# Set the parameters for the stochastic backtest
LATENCY_COMP_PARAMS = {
    0: {"prob": 0.9, "divisor": 1},
    1: {"prob": 0.9, "divisor": 1},
    2: {"prob": 0.9, "divisor": 1},
    3: {"prob": 0.9, "divisor": 1},
} # Latency compensation parameters for stochastic backtest

In [ ]:
# Set the parameters for the automated backtest
priorities = [1]
volumes = [100]

In [ ]:
# Initialize the results dictionary
results = {}

In [ ]:
# Pick the timestamp of the model to load
# ts = "2024-01-21_17-12-35" # seed 1
# ts = "2024-01-22_18-03-01" # seed 2
# ts = "2024-01-23_19-14-27" # seed 3
# ts = "2024-01-24_09-40-47" # seed 4
# ts = "2024-01-24_22-39-37" # seed 5
ts = "2024-01-24_22-39-37_best_9_297.5" # seed 5 (best model)

In [ ]:
from rl.utils import save_model, load_model

# Load the model
load_path = os.path.join(os.getcwd(), "models")
# load_path = os.path.join(os.getcwd(), "saved_models")
learner, reward_net, stats = load_model(load_path, ts)
print(f"Loaded model for timestamp: {ts}")

In [ ]:
from lob.traders import RLMarketMaker

In [ ]:
for seed in range(1, 51):
    for priority in priorities:
        for volume in volumes: 
            RNG = np.random.default_rng(seed=seed)

            # Initialize the limit order book and traders
            start = time.time()
            traders = {}

            # # Pure market making strategy
            trader_id = f"RL_prior_{priority}_vol_{volume}_{seed}"
            # inventory_manage = True
            description = f"RL market maker with priority {priority} and volume {volume}."
            
            # Set the commission model
            if volume == 100:
                if priority == 0:
                    tier = 5
                elif priority == 1:
                    tier = 5
                elif priority == 2:
                    tier = 2
                elif priority == 3:
                    tier = 1
            elif volume == 10:
                if priority == 0:
                    tier = 4
                elif priority == 1:
                    tier = 3
                elif priority == 2:
                    tier = 1
                elif priority == 3:
                    tier = 1
            
            com_model = BitCommissions(tier=tier)
            trader = RLMarketMaker(
                id=trader_id,
                com_model=com_model,
                volume=volume,
                policy=learner.policy,
            )
            traders[trader.id] = trader

            # Initialize the exchange
            exchange = Exchange(
                exchange_name=EXCHANGE_NAME,
                symbol_name=SYMBOL,
                tick_size=TICK_SIZE,
                lot_size=LOT_SIZE,
                depth=DEPTH,
                traders=traders,
                max_steps=MAX_STEPS,
                ts_start=TS_START,
                ts_end=TS_END,
                win=WIN,
                path=PATH,
                rl_trader_id=trader_id,
                latency_comp_params=LATENCY_COMP_PARAMS,
                logging=LOGGING,
                ts_save=TS_SAVE,
                description=description,
                rng=RNG,
                )
            end = round(time.time() - start, 2)

            # Run the exchange simulation
            start = time.time()
            exchange.run()
            end = round(time.time() - start, 2)

            # Save the results
            timestamps = exchange.stats["ts"]
            trader_stats = traders[trader_id].stats
            initial_cost = 20.5 * volume * 2
            results[trader_id] = {
                "timestamps": timestamps,
                "trader_stats": trader_stats,
                "initial_cost": initial_cost,
            }
        
send_notification(message="Backtest finished!", time=20000)

In [ ]:
# Save the results to a pickle file
save_dir = "automated_backtests"

ensure_dir_exists(save_dir)
save_path = os.path.join(save_dir, f"results_{TS_SAVE}.pickle")
with open(save_path, "wb") as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
print(f"Results saved to {save_path}.")